<a href="https://colab.research.google.com/github/darkzard05/Cora_dataset_benchmark/blob/main/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch-scatter torch-sparse torch-spline-conv torch_geometric -f https://data.pyg.org/whl/torch-1.11.0+cu113.html

Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 7.9 MB 5.3 MB/s 
     |████████████████████████████████| 3.5 MB 32.1 MB/s 
     |████████████████████████████████| 750 kB 30.9 MB/s 
     |████████████████████████████████| 407 kB 5.4 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=624a57cb951cd88768d9953df1e9026e76914859ae09c84e37373d808dc1620a
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [2]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear
import torch_geometric
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import SplineConv

print(torch.__version__)

1.11.0+cu113


In [3]:
dataset_name = 'Cora'
dataset = Planetoid(root='/tmp/'+dataset_name, name=dataset_name, 
                    transform=T.TargetIndegree()
                    )

Processing...
Done!


In [4]:
data = dataset[0]
print(f'dataset name: {dataset_name}')
print(f'number of nodes: {data.num_nodes} // number of edges: {data.num_edges} // number of classes: {dataset.num_classes}')
print(f'number of node features: {data.num_node_features} // number of edge features: {data.num_edge_features}')
print(data.x, data.edge_index, data.edge_attr, data.y)
print(data.train_mask.sum(), data.val_mask.sum(), data.test_mask.sum())

dataset name: Cora
number of nodes: 2708 // number of edges: 10556 // number of classes: 7
number of node features: 1433 // number of edge features: 1
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]) tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]]) tensor([[0.0179],
        [0.0238],
        [0.0179],
        ...,
        [0.1964],
        [0.0238],
        [0.0238]]) tensor([3, 4, 4,  ..., 3, 3, 3])
tensor(140) tensor(500) tensor(1000)


In [5]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

from model import appnp, sgconv, splineconv

print('cuda' if torch.cuda.is_available() else 'cpu', 'is available')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data = dataset[0].to(device)
hidden, K_sgconv, K_appnp, alpha = 16, 2, 50, 0.1
model_list, optimizer_list = [], []
criterion = torch.nn.CrossEntropyLoss()

model_list.append(appnp(dataset=dataset, hidden=hidden, K=K_appnp, alpha=alpha).to(device))
model_list.append(sgconv(dataset=dataset, K=K_sgconv).to(device))
model_list.append(splineconv(dataset=dataset, hidden=hidden).to(device))

for model in model_list:
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=5e-3)
    optimizer_list.append(optimizer)
    model.reset_parameters()
    print(model)
    print({i:list(j.shape) for i, j in model.named_parameters()})
    print('-' * 100)

cuda is available
appnp(
  (layer1): Linear(in_features=1433, out_features=16, bias=True)
  (layer2): Linear(in_features=16, out_features=7, bias=True)
  (prop): APPNP(K=50, alpha=0.1)
)
{'layer1.weight': [16, 1433], 'layer1.bias': [16], 'layer2.weight': [7, 16], 'layer2.bias': [7]}
----------------------------------------------------------------------------------------------------
sgconv(
  (layer1): SGConv(1433, 7, K=2)
)
{'layer1.lin.weight': [7, 1433], 'layer1.lin.bias': [7]}
----------------------------------------------------------------------------------------------------
splineconv(
  (layer1): SplineConv(1433, 16, dim=1)
  (layer2): SplineConv(16, 7, dim=1)
)
{'layer1.weight': [2, 1433, 16], 'layer1.bias': [16], 'layer1.lin.weight': [16, 1433], 'layer2.weight': [2, 16, 7], 'layer2.bias': [7], 'layer2.lin.weight': [7, 16]}
----------------------------------------------------------------------------------------------------


In [6]:
def train(model, optimizer):
    model.train()
    optimizer.zero_grad(set_to_none=True)
    loss = criterion(model(data)[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

def test(model, optimizer, mask):
    model.eval()
    optimizer.zero_grad(set_to_none=True)
    correct = model(data).argmax(dim=1)[mask] == data.y[mask]
    return int(correct.sum()) / int(mask.sum())

result = {}

for model, optimizer in zip(model_list, optimizer_list):
    best_acc = 0
    epochs = 1000
    for epoch in range(1, epochs+1):
        loss = train(model, optimizer)
        train_acc = test(model, optimizer, data.train_mask)
        val_acc = test(model, optimizer, data.val_mask)
        test_acc = test(model, optimizer, data.test_mask)
        if best_acc < test_acc:
            best_acc = test_acc
            print(f'epoch: {epoch} | loss: {loss.item():.4f} | train_acc: {train_acc:.4f} | val_acc: {val_acc} | test_acc: {test_acc}')
    result[model] = best_acc

print(result)

epoch: 1 | loss: 1.9461 | train_acc: 0.1429 | val_acc: 0.058 | test_acc: 0.065
epoch: 2 | loss: 1.9241 | train_acc: 0.4071 | val_acc: 0.254 | test_acc: 0.248
epoch: 3 | loss: 1.8997 | train_acc: 0.6000 | val_acc: 0.392 | test_acc: 0.43
epoch: 4 | loss: 1.8636 | train_acc: 0.6500 | val_acc: 0.49 | test_acc: 0.525
epoch: 5 | loss: 1.8280 | train_acc: 0.6714 | val_acc: 0.522 | test_acc: 0.561
epoch: 6 | loss: 1.7877 | train_acc: 0.6714 | val_acc: 0.532 | test_acc: 0.582
epoch: 7 | loss: 1.7517 | train_acc: 0.6714 | val_acc: 0.548 | test_acc: 0.591
epoch: 8 | loss: 1.6972 | train_acc: 0.6857 | val_acc: 0.554 | test_acc: 0.596
epoch: 9 | loss: 1.6529 | train_acc: 0.6857 | val_acc: 0.556 | test_acc: 0.597
epoch: 14 | loss: 1.4616 | train_acc: 0.6857 | val_acc: 0.558 | test_acc: 0.6
epoch: 15 | loss: 1.4301 | train_acc: 0.6857 | val_acc: 0.562 | test_acc: 0.602
epoch: 17 | loss: 1.3579 | train_acc: 0.6929 | val_acc: 0.562 | test_acc: 0.605
epoch: 18 | loss: 1.3026 | train_acc: 0.6929 | val_ac